In [2]:
import numpy as np
import pandas as pd
import pickle
#from build_database import flux_obj
from scipy import interpolate
import matplotlib.pyplot as plt
import os
import itertools
import random
#%matplotlib inline
# Autoload changes made in external editor:
%load_ext autoreload
%autoreload 2

# --------------- Latex Plot Beautification --------------------------
fig_width_pt = 650.0  # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (np.sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = fig_width*golden_mean      # height in inches
fig_size =  [fig_width+1,fig_height+1]
params = {'backend': 'ps',
          'axes.labelsize': 14,
          'text.fontsize': 14,
          'legend.fontsize': 10,
          'xtick.labelsize': 10,
          'ytick.labelsize': 10,
          'text.usetex': False,
          'figure.figsize': fig_size}
plt.rcParams.update(params)
# --------------- Latex Plot Beautification --------------------------

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from calc_scattering import calc_scattering
import sim_consts as sc
import time

#directory='/Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/python/'
ray_directory='/Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/rays/kp0_30sec/'
code_directory='/Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/python/c/'

center_lat=45
lower_freq=200
upper_freq=240
L_shells= 3.0 #3.0 #2.5
I0 = -100000.0
#dlat = 1


#tstart = time.time()
ray_crossings = calc_scattering(ray_directory, I0, center_lat, lower_freq, upper_freq, L_shells)
#tstop = time.time()

#print "Elapsed time: %g seconds"%(tstop - tstart)

loading  /Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/rays/kp0_30sec/newray200.dat
loading  /Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/rays/kp0_30sec/newray240.dat
Ray starting at 44 degrees
DIV_LAT_NUM: 11.0
DIV_FREQ_NUM: 2.0
Latitude interpolating steps:   [ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]
Frequency interpolating steps:  [ 0.  1.]
center_lat:  45
dlat: 1.0
dfreq: 40
MAX_POWER: 0.000866983547658
testing 37 cases (coarse grid)
(151, 22)
Ray starting at 45 degrees
DIV_LAT_NUM: 11.0
DIV_FREQ_NUM: 2.0
Latitude interpolating steps:   [ 0.   0.1  0.2  0.3  0.4  0.5  0.6  0.7  0.8  0.9  1. ]
Frequency interpolating steps:  [ 0.  1.]
center_lat:  45
dlat: 1.0
dfreq: 40
MAX_POWER: 0.000866983547658
testing 31 cases (coarse grid)
(136, 22)
Elapsed time (Interpolation and crossing detection): 0.41739 seconds


In [10]:
# Plot the equal-area slices, along with fieldlines, to show that the coordinates are right.
from calc_scattering import gen_EA_array
from matplotlib import collections  as mc
import sim_consts as sc

L_targ = L_shells
EA = gen_EA_array(L_targ)

# Initialize plot
fig, ax = plt.subplots()

# Plot the earth
earth = plt.Circle((0,0),1,color='k',alpha=0.3)

# Plot the fieldline
lam = np.linspace(-50,50,181)
L_r = L_targ*pow(np.cos(lam*sc.D2R),2)
Lx  = L_r*np.cos(lam*sc.D2R)
Ly  = L_r*np.sin(lam*sc.D2R)

# Plot the margin fieldlines
Lmr1 = (L_targ + sc.L_MARGIN)*pow(np.cos(lam*sc.D2R),2)
Lmr1x= Lmr1*np.cos(lam*sc.D2R)
Lmr1y= Lmr1*np.sin(lam*sc.D2R)
Lmr2 = (L_targ - sc.L_MARGIN)*pow(np.cos(lam*sc.D2R),2)
Lmr2x= Lmr2*np.cos(lam*sc.D2R)
Lmr2y= Lmr2*np.sin(lam*sc.D2R)

# Plot Equal-Area slices
po = zip(EA['x1'],EA['y1'])
p1 = zip(EA['x2'],EA['y2'])
points = zip(po, p1)
lc = mc.LineCollection(points)

# Plot Ray Crossing Segments
rayc = mc.LineCollection(ray_crossings['cross_coords'],colors=(0,0.5,0.5,1),linewidth=3)


plt.plot(Lx,Ly,color='r',linewidth=2)  # Field line
plt.plot(Lmr1x, Lmr1y,color='g')       # Outer boundary
plt.plot(Lmr2x, Lmr2y,color='g')       # Inner boundary
ax.add_patch(earth)                    # The earth (round)
ax.add_collection(lc)                  # Equal-Area segments
ax.add_collection(rayc)                # Crossed ray segments

ax.autoscale()
ax.set_xlim([L_targ - 2*sc.L_MARGIN,L_targ + 1.5*sc.L_MARGIN])
ax.set_ylim([-0.8,0.8])
ax.set_xlabel('L')
ax.set_ylabel('L')


# Add in the segments
from matplotlib import collections  as mc
from load_rayfile import load_rayfile
#directory = '/Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/debugging'

l_min = center_lat - sc.LAT_SPREAD/2
l_max = center_lat + sc.LAT_SPREAD/2

t_min = 0
t_max = 15




# Print the C-model crossings (if we want)
Ccross = pd.read_csv(os.path.join(code_directory, 'crossing_log.txt'),header=None,sep=' ')
Ccross.columns=['CROSSING','t','r1','r2','lat1','lat2']

X_prev = Ccross.r1*np.cos(Ccross.lat1*np.pi/180.0)
Y_prev = Ccross.r1*np.sin(Ccross.lat1*np.pi/180.0)
X_curr = Ccross.r2*np.cos(Ccross.lat2*np.pi/180.0)
Y_curr = Ccross.r2*np.sin(Ccross.lat2*np.pi/180.0)

p1 = zip(X_prev,Y_prev)
p2 = zip(X_curr,Y_curr)
points = zip(p1, p2)

cc = mc.LineCollection(points,colors=(0.5,0,0.5,1),linewidth=1, alpha=0.5)
ax.add_collection(cc)


# Plot the rays
RF = load_rayfile(ray_directory, lower_freq)

all_lats = np.array(sorted(RF.keys()))
lats = all_lats[(all_lats >= l_min) & (all_lats <= l_max)]
print all_lats

for l in lats:    
    X = RF[l].distre*np.cos(RF[l].lat[(RF[l].tg > t_min) & (RF[l].tg < t_max)]*np.pi/180.0)
    Y = RF[l].distre*np.sin(RF[l].lat[(RF[l].tg > t_min) & (RF[l].tg < t_max)]*np.pi/180.0)
    # pick colormap here
    C = plt.cm.viridis(RF[l].power)
    points = np.array([X, Y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = mc.LineCollection(segments, colors=C,alpha=0.3,linewidth=3)

    ax.add_collection(lc)


RF = load_rayfile(ray_directory, upper_freq)
for l in lats:    
    X = RF[l].distre*np.cos(RF[l].lat[(RF[l].tg > t_min) & (RF[l].tg < t_max)]*np.pi/180.0)
    Y = RF[l].distre*np.sin(RF[l].lat[(RF[l].tg > t_min) & (RF[l].tg < t_max)]*np.pi/180.0)
    # pick colormap here
    C = plt.cm.viridis(RF[l].power)
    points = np.array([X, Y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    lc = mc.LineCollection(segments, colors=C,alpha=0.3,linewidth=3)
#     lc.set_linewidth(10)

    ax.add_collection(lc)
    

ax.autoscale()
# ax.set_xlim([2.5,3.5])
# ax.set_ylim([-0.2, 1])
ax.set_xlabel('L')
ax.set_ylabel('L')



plt.show()





loading  /Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/rays/kp0_30sec/newray200.dat
[  7.   8.   9.  10.  11.  12.  13.  14.  15.  16.  17.  18.  19.  20.  21.
  22.  23.  24.  25.  26.  27.  28.  29.  30.  31.  32.  33.  34.  35.  36.
  37.  38.  39.  40.  41.  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.
  52.  53.  54.  55.  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.
  67.  68.  69.  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.]
loading  /Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/rays/kp0_30sec/newray240.dat


In [ ]:
print ray_crossings['cross_coords'].shape

from calc_scattering import calc_resonant_pitchangle_change
from calc_scattering import gen_EA_array
from calc_scattering import get_flight_time_constant

L_targ = L_shells
EA_array = gen_EA_array(L_targ)

# print EA_array
DA_N, DA_S = calc_resonant_pitchangle_change(ray_crossings, L_targ)


In [ ]:
tvec = np.linspace(0, sc.T_MAX,sc.NUM_STEPS)

ax1 = plt.subplot(2,1,1)
plt.pcolor(tvec, (sc.E_tot_arr*1e-6), DA_N)
#plt.imshow(DA_N)
plt.ylabel('Energy (MeV)')

ax2 = plt.subplot(2,1,2)
plt.pcolor(tvec, (sc.E_tot_arr*1e-6), DA_S)
#plt.imshow(DA_S)
plt.xlabel('Time (sec)')
plt.ylabel('Energy (MeV)')
plt.show()

In [ ]:


print np.max(DA_N)
print np.max(DA_S)
print np.sum(DA_N!=0)


a,b = np.where(DA_N==np.max(DA_N))
#print a, b
#print DA_N[a,b]

print DA_N[a,b]
# plt.figure()

# plt.plot(DA_N[a,:])
# plt.show

In [ ]:
from load_rayfile import load_rayfile
rlf = load_rayfile('/Users/austin/FUSE/shared/users/asousa/WIPP/WIPPy/rays/full_kp0_15sec/',1000)
print sorted(rlf.keys())

In [ ]:
dlat = 1
dfreq = 50

DIV_LAT_NUM = np.ceil(dlat/sc.LAT_STEP + 1)
DIV_FREQ_NUM = np.ceil(dfreq/sc.F_STEP + 1)
lat_fine_grid = np.linspace(0, 1, DIV_LAT_NUM)
freq_fine_grid= np.linspace(0, 1, DIV_FREQ_NUM)
print "Latitude interpolating steps:  ", lat_fine_grid
print "Frequency interpolating steps: ", freq_fine_grid
interp_grid = []

import itertools

#tmp = np.array(itertools.product(lat_fine_grid, freq_fine_grid))
tmp = np.array([(x,y) for x in lat_fine_grid for y in freq_fine_grid])
#tmp = np.array(zip(np.tile(lat_fine_grid, DIV_FREQ_NUM), np.tile(freq_fine_grid, DIV_LAT_NUM)))
for l in lat_fine_grid:
    for f in freq_fine_grid:
        interp_grid.append([l, f])

interp_grid = np.array(interp_grid)
print tmp
print interp_grid


fine_grid_size = DIV_LAT_NUM*DIV_FREQ_NUM

In [ ]:
print [(k, len(rlf[k])) for k in rlf.keys() ]